In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import SimulationData as sd
import RFTrackTools as rfttools
import copy
import json

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
plt.rcParams['figure.figsize'] = [9.6, 6.4]
defaultColorCycle = plt.rcParams["axes.prop_cycle"].by_key()['color']
# plotFont = {
#     'family' : 'sans-serif',
#     'weight' : 'normal',
#     'size'   : 12
# }
# matplotlib.rc('font', **plotFont)
# plt.rc('legend', fontsize=10)

# RF-Track, Positron Linac V0

## Simple Analytical Tracking of Longitudinal Phaase Space

### Distribution at the End of Positron Linac

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/CaptureLinacUpTo200MeV_LBandLargeR_RealSolenoids_Type1and2_TargetAt35mm/DistrOut_PositronLinac.dat'
beamEndAnalytical, _ = bd.convert_rftrack_to_standard_df(
    sourceFilePath=DISTR_REL_PATH, sourceFormat='octave', octaveMatrixName='A_PL', rftrackDfFormat='rftrack_xp_t', pdgId=-11
)
beamEndAnalytical.describe()

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
distrList = [beamEndAnalytical]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
tMinPlot = 54.86  # [ns]
tWindowPlot = 0.15  # [ns]
plotDefs[3]['lims1'] = [tMinPlot, tMinPlot+tWindowPlot]
pzMinPlot = 1100.  # [MeV/c]
pzWindowPlot = 500.  # [Mev/c]
plotDefs[3]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
_ = bd.plot_distr(distrList, plotDefs)

## Variant 1a: Solenoids up to Damping Ring at 1.54 GeV, With Tuning Solenoid After AMD

### Intermediate Distribution After 5 RF Structures (End of First Tracking)

In [ ]:
# DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid/DistrOut_After1stTracking_6d.sdf_txt'
DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid_Ecut40MeV/DistrOut_After1stTracking_6d.sdf_txt'
beamInterm1 = bd.load_standard_fwf(DISTR_REL_PATH)
beamInterm1.describe()

In [ ]:
# DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid/DistrOut_FrontBuckets_After1stTracking_6d.sdf_txt'
DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid_Ecut40MeV/DistrOut_FrontBuckets_After1stTracking_6d.sdf_txt'
beamInterm1Front = bd.load_standard_fwf(DISTR_REL_PATH)
beamInterm1Front.describe()

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT', 'r-pz']

# distrList = [beamInterm1, beamInterm1Front]
# legendLabels=['All RF buckets', 'Front RF bucket']
# markerStyles = ['o', 'x']
# or
distrList = [beamInterm1Front]
legendLabels = None
markerStyles = None
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
tMinPlot = 54.88  # [ns]
plotDefs[3]['lims1'] = [tMinPlot, tMinPlot+tWindowPlot]
pzMinPlot = 180.  # [MeV/c]
plotDefs[3]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot*200./1540.]
rMaxPlot = 30.
plotDefs[5]['lims1'] = [0, rMaxPlot]
plotDefs[5]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot*200./1540.]
_ = bd.plot_distr(
    distrList,
    plotDefs, markerStyle=markerStyles, legendLabels=legendLabels
)

### Distribution at the End of Positron Linac (After 28 RF Structures)

In [ ]:
# DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid/DistrOut_After2ndTracking_6d.sdf_txt'
DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid_Ecut40MeV/DistrOut_After2ndTracking_6d.sdf_txt'
beamEnd1 = bd.load_standard_fwf(DISTR_REL_PATH)
beamEnd1.describe()

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT', 'r-pz']
distrList = [beamEnd1]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
tMinPlot = 306.81  # [ns]
plotDefs[3]['lims1'] = [tMinPlot, tMinPlot+tWindowPlot]
pzMinPlot = 1100.  # [MeV/c]
plotDefs[3]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
plotDefs[5]['lims1'] = [0, rMaxPlot]
plotDefs[5]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
_ = bd.plot_distr(distrList, plotDefs)

### Transport

In [ ]:
reload(rfttools)

In [ ]:
# SIM_REL_PATH_1 = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid'
SIM_REL_PATH_1 = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid_Ecut40MeV'
totParticlesAtSimStart = 136506
normFactorCaptureEff2 = beamInterm1Front.shape[0] / totParticlesAtSimStart

# fig1, ax1 = plt.subplots(7, 1, figsize=(9.6, 14.))
# or
# fig1, ax1 = plt.subplots(3, 1, figsize=(9.6, 6.4))
# or
fig1, ax1l = plt.subplots(3, 1, figsize=(10.5, 5.5))
ax1r = []
for ax in ax1l:
    ax1r.append(ax.twinx())
ax1 = [None] * (len(ax1l) + len(ax1r))
ax1[::2] = ax1l
ax1[1::2] = ax1r
quantitiesToPlot = ['Bz', None, 'mean_E', 'CaptureEfficiency', 'Sigmas', None]
rfttools.load_plot_transport(
    ax1, SIM_REL_PATH_1, fileSuffix='1', quantitiesToPlot=quantitiesToPlot)
rfttools.load_plot_transport(
    ax1, SIM_REL_PATH_1, fileSuffix='2', quantitiesToPlot=quantitiesToPlot,
    normFactorCaptureEff=normFactorCaptureEff2)
# ax1[0].legend(['Volume + Lattice', 'Volume'], loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=2)
# _ = ax1[0].set_xlim([40., 50.])
_ = ax1[0].set_ylim([0., 1.])
_ = ax1[4].set_ylim([0., 30.])
_ = ax1[4].set_xlabel('s [m]')

## Variant 1b: Solenoids up to Damping Ring at 1.54 GeV, No Tuning Solenoid After AMD

### Intermediate Distribution After 5 RF Structures (End of First Tracking)

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2/DistrOut_After1stTracking_6d.sdf_txt'
beamInterm1b = bd.load_standard_fwf(DISTR_REL_PATH)
beamInterm1b.describe()

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2/DistrOut_FrontBuckets_After1stTracking_6d.sdf_txt'
beamInterm1bFront = bd.load_standard_fwf(DISTR_REL_PATH)
beamInterm1bFront.describe()

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']

distrList = [beamInterm1b, beamInterm1bFront]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
_ = bd.plot_distr(
    distrList,
    plotDefs, markerStyle=['o', 'x'], legendLabels=['All RF buckets', 'Front RF bucket']
)

### Distribution at the End of Positron Linac (After 27 RF Structures)

In [ ]:
# DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2/DistrOut_After2ndTracking_6d.sdf_txt'
DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2/DistrOut_After2ndTracking_6d.sdf_txt'
beamEnd1b = bd.load_standard_fwf(DISTR_REL_PATH)
beamEnd1b.describe()

## Comparison Variant 1a vs. 1b

### Intermediate Distribution

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
distrList = [beamInterm1, beamInterm1b]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
_ = bd.plot_distr(distrList, plotDefs, legendLabels=['With tuning solenoid', 'Without tuning solenoid'])

### Final Distribution

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
distrList = [beamEnd1, beamEnd1b]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
_ = bd.plot_distr(distrList, plotDefs, legendLabels=['With tuning solenoid', 'Without tuning solenoid'])

### Transport

In [ ]:
reload(rfttools)

In [ ]:
SIM_REL_PATH_1 = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2_TuningSolenoid'
SIM_REL_PATH_2 = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_LBandLargeR_RealSolenoids_Type1and2'
totParticlesAtSimStart = 136506
normFactorCaptureEff2 = beamInterm1Front.shape[0] / totParticlesAtSimStart
normFactorCaptureEff2TuningSolenoid = beamInterm1bFront.shape[0] / totParticlesAtSimStart

fig1, ax1 = plt.subplots(7, 1, figsize=(9.6, 15.))
# fig1, ax1l = plt.subplots(3, 1, figsize=(9.6, 6.4))
# ax1r = []
# for ax in ax1l:
#     ax1r.append(ax.twinx())
# ax1 = [None] * (len(ax1l) + len(ax1r))
# ax1[::2] = ax1l
# ax1[1::2] = ax1r
rfttools.load_plot_transport(ax1, SIM_REL_PATH_1, fileSuffix='1')
rfttools.load_plot_transport(ax1, SIM_REL_PATH_1, fileSuffix='2', normFactorCaptureEff=normFactorCaptureEff2)
rfttools.load_plot_transport(ax1, SIM_REL_PATH_2, fileSuffix='1')
rfttools.load_plot_transport(ax1, SIM_REL_PATH_2, fileSuffix='2', normFactorCaptureEff=normFactorCaptureEff2TuningSolenoid)
# ax1[0].legend(['Volume + Lattice', 'Volume'], loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=2)
# _ = ax1[2].set_ylim([150., 250.])

## Variant 1a with Constant Ez Above 200 MeV

### Distribution at the End of Positron Linac (After 5 RF Structures + 23 Sections with Ez = 20 MV/m)

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_ConstEzAfter200MeV_RealSolenoids_Type1and2_TuningSolenoid_Ecut40MeV/DistrOut_After2ndTracking_6d.sdf_txt'
beamEnd1bConstEz = bd.load_standard_fwf(DISTR_REL_PATH)
beamEnd1bConstEz.describe()

In [ ]:
reload(bd)
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT', 'r-pz']
distrList = [beamEnd1bConstEz]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
tMinPlot = 306.85  # [ns]
plotDefs[3]['lims1'] = [tMinPlot, tMinPlot+tWindowPlot]
pzMinPlot = 1500.  # [MeV/c]
plotDefs[3]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
plotDefs[5]['lims1'] = [0, rMaxPlot]
plotDefs[5]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
_ = bd.plot_distr(distrList, plotDefs)

### Transport

In [ ]:
reload(rfttools)
SIM_REL_PATH_1 = '../../Data/RFTrack/CaptureLinac/PositronLinacUpTo1p54GeV_ConstEzAfter200MeV_RealSolenoids_Type1and2_TuningSolenoid_Ecut40MeV'
totParticlesAtSimStart = 136506
normFactorCaptureEff2 = 2815 * 30 / totParticlesAtSimStart
# normFactorCaptureEff2 = 1.

# fig1, ax1 = plt.subplots(7, 1, figsize=(9.6, 15.))
# or
fig1, ax1l = plt.subplots(3, 1, figsize=(10.5, 5.5))
ax1r = []
for ax in ax1l:
    ax1r.append(ax.twinx())
ax1 = [None] * (len(ax1l) + len(ax1r))
ax1[::2] = ax1l
ax1[1::2] = ax1r
quantitiesToPlot = ['Ez', None, 'mean_E', 'CaptureEfficiency', 'Sigmas', None]
rfttools.load_plot_transport(ax1, SIM_REL_PATH_1, fileSuffix='1', quantitiesToPlot=quantitiesToPlot)
rfttools.load_plot_transport(ax1, SIM_REL_PATH_1, fileSuffix='2', quantitiesToPlot=quantitiesToPlot, normFactorCaptureEff=normFactorCaptureEff2)
# ax1[0].legend(['Volume + Lattice', 'Volume'], loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=2)
# _ = ax1[0].set_xlim([40., 50.])
# _ = ax1[0].set_ylim([0., 1.])
_ = ax1[4].set_ylim([0., 30.])
_ = ax1[4].set_xlabel('s [m]')

## Variant 2: Solenoids Up To 780 MeV + Conventional FODO Up To Damping Ring

### Intermediate Distribution After 15 RF Structures (End of Solenoid Section)

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrack/CaptureLinac/PositronLinac_15RFStruct_LBandLargeR_SolenoidsType1and2/DistrOut_After2ndTracking_6d.sdf_txt'
beamInterm2 = bd.load_standard_fwf(DISTR_REL_PATH)
beamInterm2.describe()

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT', 'r-pz']
distrList = [beamInterm2]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
plotDefs[5]['lims1'] = [0, rMaxPlot]
pzMinPlot = 350.  # [MeV/c]
pzWindowPlot = 500.  # [MeV/c]
plotDefs[5]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
_ = bd.plot_distr(distrList, plotDefs)

### Distribution at the End of Positron Linac (After 15+13=28 RF Structures)

In [ ]:
DISTR_REL_PATH = '../../Data/RFTrack/PositronLinacSection3/PositronLinacUpTo1p54GeV_LBandLargeR_Matching2_Fodo2/DistrOut_6d.sdf_txt'
beamEnd2 = bd.load_standard_fwf(DISTR_REL_PATH)
beamEnd2.describe()

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT', 'r-pz']
distrList = [beamEnd2]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
tMinPlot = 418.9  # [ns]
plotDefs[3]['lims1'] = [tMinPlot, tMinPlot+tWindowPlot]
pzMinPlot = 1100.  # [MeV/c]
plotDefs[3]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
plotDefs[5]['lims1'] = [0, rMaxPlot]
plotDefs[5]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
_ = bd.plot_distr(distrList, plotDefs)

### Transport

In [ ]:
reload(rfttools)
SIM_REL_PATH_1 = '../../Data/RFTrack/CaptureLinac/PositronLinac_15RFStruct_LBandLargeR_SolenoidsType1and2'
SIM_REL_PATH_2 = '../../Data/RFTrack/PositronLinacSection3/PositronLinacUpTo1p54GeV_LBandLargeR_Matching2_Fodo2'
sShiftGlobal2 = 49.831  # [m]
totParticlesAtSimStart = 136506
normFactorCaptureEff1 = beamInterm1.shape[0] / totParticlesAtSimStart
normFactorCaptureEff2 = beamInterm2.shape[0] / totParticlesAtSimStart
# normFactorCaptureEff2 = 1.

# fig1, ax1 = plt.subplots(7, 1, figsize=(9.6, 14.))
# or
# fig1, ax1 = plt.subplots(3, 1, figsize=(9.6, 6.4))
# or
fig1, ax1l = plt.subplots(3, 1, figsize=(10.5, 5.5))
ax1r = []
for ax in ax1l:
    ax1r.append(ax.twinx())
ax1 = [None] * (len(ax1l) + len(ax1r))
ax1[::2] = ax1l
ax1[1::2] = ax1r
quantitiesToPlot = ['Bz', None, 'mean_E', 'CaptureEfficiency', 'Sigmas', None]
rfttools.load_plot_transport(
    ax1, SIM_REL_PATH_1, fileSuffix='2', quantitiesToPlot=quantitiesToPlot, normFactorCaptureEff=normFactorCaptureEff1)
rfttools.load_plot_transport(
    ax1, SIM_REL_PATH_2, quantitiesToPlot=quantitiesToPlot, sShiftGlobal=sShiftGlobal2, normFactorCaptureEff=normFactorCaptureEff2)
# ax1[0].legend(['Volume + Lattice', 'Volume'], loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=2)
# _ = ax1[0].set_xlim([40., 50.])
_ = ax1[0].set_ylim([0., 1.])
_ = ax1[4].set_ylim([0., 30.])
_ = ax1[4].set_xlabel('s [m]')

## Variant 3: Solenoids Up To 780 MeV + Conventional FODO Up To Damping Ring at 2.84 GeV

### Distribution at the End of Positron Linac (After 15+35=50 RF Structures)

In [ ]:
DISTR_REL_PATH = '../../RFTrackOutput/LatestSimPositronLinacSection3/DistrOut_6d.sdf_txt'
beamEnd3 = bd.load_standard_fwf(DISTR_REL_PATH)
beamEnd3.describe()

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT', 'r-pz']
distrList = [beamEnd3]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
tMinPlot = 741.92  # [ns]
plotDefs[3]['lims1'] = [tMinPlot, tMinPlot+tWindowPlot]
pzMinPlot = 2400.  # [MeV/c]
plotDefs[3]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
plotDefs[5]['lims1'] = [0, rMaxPlot]
plotDefs[5]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
_ = bd.plot_distr(distrList, plotDefs)

# Yield at Damping Ring

## Simple Analytical Tracking of Longitudinal Phase Space

In [ ]:
totElectronsDriveBeamAnalytical = 1e4
pzWindowCenterAnalytical = 1530.  # [MeV/c]
pzWindowAmplitudeAnalytical = 0.038
drAcceptanceAnalytical = {
    "t": [54.8, 55.2],  # [ns]
    "pz": [pzWindowCenterAnalytical*(1-pzWindowAmplitudeAnalytical), pzWindowCenterAnalytical*(1+pzWindowAmplitudeAnalytical)]
}
beamAcceptedAnalytical = bd.filter_distr(beamEndAnalytical, drAcceptanceAnalytical)
print('Positron yield at damping ring = {:.2f}'.format(beamAcceptedAnalytical.shape[0]/totElectronsDriveBeamAnalytical))

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
distrList = [beamEndAnalytical, beamAcceptedAnalytical]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
tMinPlot = 54.86  # [ns]
plotDefs[3]['lims1'] = [tMinPlot, tMinPlot+tWindowPlot]
pzMinPlot = 1100.  # [MeV/c]
plotDefs[3]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
_ = bd.plot_distr(distrList, plotDefs)

## Variant 1a

In [ ]:
totElectronsDriveBeam1 = 1e4
pzWindowCenter1 = 1505.  # [MeV/c]
pzWindowAmplitude1 = 0.038
drAcceptance1 = {
    "t": [306.7, 307.1],  # [ns]
    "pz": [pzWindowCenter1*(1-pzWindowAmplitude1), pzWindowCenter1*(1+pzWindowAmplitude1)]
}
beamAccepted1 = bd.filter_distr(beamEnd1, drAcceptance1)
print('Positron yield at damping ring = {:.2f}'.format(beamAccepted1.shape[0]/totElectronsDriveBeam1))

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
distrList = [beamEnd1, beamAccepted1]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
plotDefs[1]['lims2'] = [-5., 5.]  # [mrad]
tMinPlot = 306.81  # [ns]
plotDefs[3]['lims1'] = [tMinPlot, tMinPlot+tWindowPlot]
pzMinPlot = 1100.  # [MeV/c]
plotDefs[3]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
_ = bd.plot_distr(distrList, plotDefs)

## Variant 2

In [ ]:
totElectronsDriveBeam2 = 1e4
pzWindowCenter2 = 1510.  # [MeV/c]
pzWindowAmplitude2 = 0.038
drAcceptance2 = {
    "t": [418.8, 419.1],  # [ns]
    "pz": [pzWindowCenter2*(1-pzWindowAmplitude2), pzWindowCenter2*(1+pzWindowAmplitude2)]
}
beamAccepted2 = bd.filter_distr(beamEnd2, drAcceptance2)
print('Positron yield at damping ring = {:.2f}'.format(beamAccepted2.shape[0]/totElectronsDriveBeam2))

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
distrList = [beamEnd2, beamAccepted2]
plotDefs = bd.set_plot_defs_from_distrs(distrList, setNames=plotSets)
tMinPlot = 418.9  # [ns]
plotDefs[3]['lims1'] = [tMinPlot, tMinPlot+tWindowPlot]
pzMinPlot = 1100.  # [MeV/c]
plotDefs[3]['lims2'] = [pzMinPlot, pzMinPlot+pzWindowPlot]
_ = bd.plot_distr(distrList, plotDefs)

<div class="alert alert-block alert-success">
Some good news.
</div>

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some danger.
</div>